# Analysis

In [ ]:
import numpy as np
import sympy as sp
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, vlatex, vprint, find_dynamicsymbols
from sympy import symbols, Function, sin, cos, Matrix, pi, re, im
from IPython.display import display, Markdown, Math
init_vprinting()

In [ ]:
Damping = False
i = sp.I
L, Lg, Jg, m1, m2, m3, lamda, t = symbols('L Lg Jg m1 m2 m3 lamda t')
k1, k2, k3, k4, k5, kt3 = symbols('k1 k2 k3 k4 k5 kt3')
c1, c2, c3, c4, c5, ct3 = symbols('c1 c2 c3 c4 c5 ct3')
x0, x1, x2, x3, y3, theta = dynamicsymbols('x0 x1 x2 x3 y3 theta')

# Optie 1 is zoals we hem hadden en ik heb een nieuwe optie toegevoegd waar theta in rust als 0 is gedfinieerd. Het geeft hetzelfde resultaat

x4 = x3 + L * cos(theta)
y4 = y3 + L * sin(theta)
x3g = x3 + Lg * cos(theta)
y3g = y3 + Lg * sin(theta)


In [ ]:
dx0 = sp.diff(x0, t)
dx1 = sp.diff(x1, t)
dx2 = sp.diff(x2, t)
dx3 = sp.diff(x3, t)
dy3 = sp.diff(y3, t)
dx3g = sp.diff(x3g, t)
dy3g = sp.diff(y3g, t)
dx4 = sp.diff(x4, t)
dy4 = sp.diff(y4, t)
dTheta = sp.diff(theta, t)

ddx0 = sp.diff(dx0, t)
ddx1 = sp.diff(dx1, t)
ddx2 = sp.diff(dx2, t)
ddx3 = sp.diff(dx3, t)
ddy3 = sp.diff(dy3, t)
ddTheta = sp.diff(dTheta, t)

q = Matrix([x1, x2, x3, y3, theta])
qDot = Matrix([dx1, dx2, dx3, dy3, dTheta])
qDDot = Matrix([ddx1, ddx2, ddx3, ddy3, ddTheta])

dq_zero = {
    dx1: 0,
    dx2: 0,
    dx3: 0,
    dy3: 0,
    dTheta: 0}

ddq_zero = {
    ddx1: 0,
    ddx2: 0,
    ddx3: 0,
    ddy3: 0,
    ddTheta: 0}

In [ ]:
values = {
    m1: 0.45,
    m2: 1.15,
    m3: 1.9,
    Jg: 0.0149,
    k1: 155.8*10**3,
    k2: 23.6*10**3,
    k3: 444.6*10**3,
    k4: 415.4*10**3,
    k5: 50.25*10**3,
    kt3: 2,
    c1: 30,
    c2: 202.84,
    c3: 500,
    c4: 164.59,
    c5: 49.99,
    ct3: 4.9,
    L: 0.298,
    Lg: 0.6 * 0.298,
}

dynamicValues = {
    theta: pi/2,
    y3: 0,
    dy3: 0,
    ddy3: 0,
    dTheta: 0,
    ddTheta: 0,
    x1: 0,
    x2: 0,
    x3: 0
}

# Energies

In [ ]:
T = m1/2 * dx1 ** 2 + \
    m2/2 * dx2 ** 2 + \
    m3/2 * dx3g ** 2 + \
    m3/2 * dy3g ** 2 + \
    Jg/2 * dTheta ** 2
display(Math("""\mathbf{T} = """ + vlatex(T)))

In [ ]:
D = c1 / 2 * (dx0 - dx1) ** 2 + \
    c2 / 2 * (dx1 - dx2) ** 2 + \
    c3 / 2 * (dx2 - dx3) ** 2 + \
    c4 / 2 * dx4 ** 2 + \
    c5 / 2 * dy4 ** 2 + \
    ct3 / 2 * dTheta ** 2

display(Math("""\mathbf{D} = """ + vlatex(D)))

In [ ]:
V = k1 / 2 * (x0 - x1) ** 2 + \
    k2 / 2 * (x1 - x2) ** 2 + \
    k3 / 2 * (x2 - x3) ** 2 + \
    k4 / 2 * (x4-L*cos(pi/2)) ** 2 + \
    k5 / 2 * (y4-L*sin(pi/2)) ** 2 + \
    kt3 / 2 * (theta-pi/2) ** 2

display(Math("""\mathbf{V} = """ + vlatex(V)))

# EoM with Lagrange

In [ ]:
fT = (T.diff(qDot).diff(t) - T.diff(q)).simplify()
fV = V.diff(q).simplify()
fD = D.diff(qDot).simplify()
Q_cons = (-V.diff(q) + D.diff(qDot)).simplify()

display(Math("""\mathbf{f_T} = """ + vlatex(fT)))
display(Math("""\mathbf{f_V} = """ + vlatex(fV)))
display(Math("""\mathbf{f_D} = """ + vlatex(fD)))
display(Math("""\mathbf{Q} = """ + vlatex(Q_cons)))

In [ ]:
EoM = fT + fV + fD
for j in EoM:
    display(j)

In [ ]:
qEq = Q_cons.xreplace(dynamicValues).simplify()
display(Math("""\mathbf{Q_{eq}} = """ + vlatex(qEq)))

# Linearization

In [ ]:
kMatrix = (fT + fV).jacobian(q)
kMatrix_alt = fV.jacobian(q)# V.diff(q).jacobian(q)
cMatrix = (fT + fD).jacobian(qDot)
mMatrix = fT.jacobian(qDDot)

kDotsReplace = kMatrix.xreplace(dq_zero).xreplace(ddq_zero)    # is hetzelfde als fV.jacobian(q)
kDotsReplace_alt = kMatrix_alt.xreplace(dq_zero).xreplace(ddq_zero)
cDotsReplace = cMatrix.xreplace(dq_zero).xreplace(ddq_zero)
mDotsReplace = mMatrix.xreplace(dq_zero).xreplace(ddq_zero)    # is hetzelfde als T.diff(qDot).jacobian(qDot)

kLinear = kDotsReplace.xreplace(dynamicValues)
kLinear_alt = kDotsReplace_alt.xreplace(dynamicValues)
cLinear = cDotsReplace.xreplace(dynamicValues)
mLinear = mDotsReplace.xreplace(dynamicValues)

In [ ]:
display(Math("""\mathbf{K_{Linear}} = """ + vlatex(kLinear)))

#print(vlatex(M))

In [ ]:
display(Math("""\mathbf{C_{Linear}} = """ + vlatex(cLinear)))

In [ ]:
display(Math("""\mathbf{M_{Linear}} = """ + vlatex(mLinear)))

# Evaluation

In [ ]:
I = sp.eye(mMatrix.shape[0], mMatrix.shape[1])
zeroMatrix = sp.zeros(mMatrix.shape[0], mMatrix.shape[1])

In [ ]:
K = sp.simplify(kLinear.xreplace(values))
K = K.applyfunc(sp.nsimplify)
K

In [ ]:
C = sp.simplify(cLinear.xreplace(values))
C = C.applyfunc(sp.nsimplify)
C

In [ ]:
M = sp.simplify(mLinear.xreplace(values))
M = M.applyfunc(sp.nsimplify)
M

In [ ]:
# Undamped
Eqk = (K - lamda**2*M)
Lk = sp.solve(sp.Eq(Eqk.det(), 0))
Lk = sp.Matrix(Lk).evalf()
display(Math("""\mathbf{\lambda_{undamped}} = """ + vlatex(Lk)))

eigenVector = [Eqk.xreplace({lamda: Lk[1],}).eigenvects()]
eigenVector.append(Eqk.xreplace({lamda: Lk[3],}).eigenvects())
eigenVector.append(Eqk.xreplace({lamda: Lk[5],}).eigenvects())
eigenVector.append(Eqk.xreplace({lamda: Lk[7],}).eigenvects())
eigenVector.append(Eqk.xreplace({lamda: Lk[9],}).eigenvects())

eigenvecs = []
for i in eigenVector:
    for j in i:
        if abs(j[0]) < 1e-8:
            eigenvecs.append(j[2][0].T)

eigenVectors = sp.Matrix(eigenvecs).T

x = [x1, x2, x3, y3, theta]
eigenVectors

In [ ]:
# Damped
Eqs = (lamda**2*M+lamda*C+K)
Zs = sp.solve(sp.Eq(Eqs.det(), 0))
Zs = sp.Matrix(Zs).evalf()
display(Math("""\mathbf{\lambda_{damped}} = """ + vlatex(Zs)))


eigenVectorsD = np.empty((0,5))
eigenVectorsZero = Matrix([])
for r in range(len(Zs)):
    eigenVectorD = [Eqs.xreplace({lamda: Zs[r],}).eigenvects()]
    for i in eigenVectorD:
        for j in i:
            if abs(j[0]) < 1e-8:
                listJ = []
                for x in range(len(j[2][0])):
                    if abs(j[2][0][x]) < 1e-8:
                        listJ.append(0)
                    else:
                        listJ.append(j[2][0][x])
                eigenVectorsZero = eigenVectorsZero.col_insert(-1, Matrix([x for x in listJ]))
                
                eigenVectorsD = np.concatenate((eigenVectorsD,j[2][0].T))
eigenVectorsD = Matrix(eigenVectorsZero)

display(sp.N(eigenVectorsD, 5))


# Eigenfrequency and eigenmodes

### Undamped

In [ ]:
# Undamped eigenfrequency and eigenmodes
eigfreq_ud = np.array([Lk[1],Lk[3],Lk[5],Lk[7],Lk[9]]).T
display(Math("""\mathbf{\omega_{r}} = """ + vlatex(eigfreq_ud)))
eigmode_ud = np.array(eigenVectors)
display(Math("""\mathbf{x_{r}} = """ + vlatex(sp.Matrix(eigmode_ud))))

In [ ]:
# check for rigid body modes. There are none
for j in eigmode_ud.T:
    display(np.array(K).dot(j))


In [ ]:
# generalized stiffness (4.18)
gamma_r = [(x.T).dot(K).dot(x) for x in eigmode_ud.T]
display(Math("""\mathbf{\gamma_{r}} = """ + vlatex(gamma_r)))

# generalized mass (4.19)
mu_r = [(x.T).dot(M).dot(x) for x in eigmode_ud.T]
display(Math("""\mathbf{\mu_{r}} = """ + vlatex(mu_r)))

In [ ]:
# modal components intitial conditions (4.24)
q_zero = np.array([0,0,0,0,0]).T
dq_zero = np.array([0,0,0,0,0]).T
eta_r0 = [(x.T).dot(M).dot(q_zero) for x in eigmode_ud.T]
display(Math("""\mathbf{\eta_{r}}(0) = """ + vlatex(eta_r0)))
deta_r0 = [(x.T).dot(M).dot(dq_zero) for x in eigmode_ud.T]
display(Math("""\mathbf{\dot{\eta}_{r}}(0) = """ + vlatex(deta_r0)))

# inverse laplace transform hr (4.28)
h_r =[sin(w*t)/w for w in eigfreq_ud]
display(Math("""\mathbf{h_{r}}(t) = """ + vlatex(sp.Matrix(h_r))))

# external forces from q matrix
s,w = symbols('s w')
x0_def = s*sin(w*t)
pt = np.array([[k1*x0_def],[0],[0],[0],[0]])

# modal participation factor (4.23)
phi_r = []
for j in range(5):
    phi = (eigmode_ud[:,j].T).dot(pt) / mu_r[j]
    phi_r.append(phi[0])
display(Math("""\mathbf{\phi_{r}}(t) = """ + vlatex(sp.Matrix(phi_r))))

# solution for eta_r (4.29)
tau = symbols('tau')
eta_r = []
for j in range(5):
    duhamel_f = phi_r[j].xreplace({t:tau})*h_r[j].xreplace({t:(t-tau)})
    duhamel_integral = sp.integrate(duhamel_f,(tau,0,t))
    eta = eta_r0[j]*cos(eigfreq_ud[j]*t) + deta_r0[j]*h_r[j] + duhamel_integral
    eta_r.append(sp.simplify(eta))
display(Math("""\mathbf{\eta_{r}}(t) = """ + vlatex(sp.Matrix(eta_r))))

# system response q(t) (4.21)
q_res_ud = np.zeros(5,)
for j in range(5):
    q_t = eta_r[j]*eigmode_ud[:,j]
    q_res_ud = q_res_ud + q_t
q_res_ud = sp.Matrix(q_res_ud)

In [ ]:
# System response
#w_value = np.double(eigfreq_ud[2]+0.001)
w_value = 230
s_value = 0.005
x0_values = {w:w_value,
             s:s_value}

q_t_ud = q_res_ud.xreplace(x0_values).xreplace(values)
t1 = 0
t2 = 0.5
delta_t = 0.001
t_num = np.linspace(t1,t2-delta_t,int((t2-t1)/delta_t))
q_num = np.array([q_t_ud.xreplace({t:ti}) for ti in t_num]).T.astype(np.float64)

In [ ]:
# visualisation
def plot_gen_coords(q,t_num,w):
    plt.plot(t_num,q[0],label=r'$x1$')
    plt.plot(t_num,q[1],label=r'$x2$')
    plt.plot(t_num,q[2],label=r'$x3$')
    plt.plot(t_num,q[3],label=r'$y3$')
    plt.plot(t_num,q[4],label=r'$\theta_3$')
    plt.legend()
    plt.title("System response for " + r'$\omega = $' + str(w) + ' rad/s')
    plt.xlabel('Time (s)')
    plt.ylabel('q')

def animate_system(q,t_num,w,s):
    scale = 0.2
    m_size = scale
    spring_size = scale
    
    x = q[2]
    yaw = q[4]
    fig, ax = plt.subplots()
    fig.set_size_inches((6.0, 4.0))
    ax.set_aspect('equal')
    ax.set_xlim(-1.5*scale, 6*scale)
    ax.set_ylim(-scale, 4*scale)
    
    m1_0 = (0,0)
    m2_0 = (m_size+spring_size, 0)
    m3_0 = (2*(m_size+spring_size), m_size/2)
    
    m1 = patches.Rectangle((m1_0[0], m1_0[1]), m_size, m_size, fc='grey')
    m2 = patches.Rectangle((m2_0[0], m2_0[1]), m_size, m_size, fc='grey')
    m3 = patches.ConnectionPatch((m3_0[0], m3_0[1]), (m3_0[0], m3_0[1]+2*m_size), coordsA=ax.transData,ec='grey',linewidth=5)
    
    g1 = patches.Circle((m1_0[0]-spring_size, m1_0[0] + m_size/2), radius=scale/20,fc='black')
    g2 = patches.Circle((m3_0[0], m3_0[1]+2*m_size+spring_size), radius=scale/20,fc='black')
    g3 = patches.Circle((m3_0[0]+spring_size, m3_0[1]+m_size*2), radius=scale/20,fc='black')
    k1 = patches.ConnectionPatch((m1_0[0]-spring_size, m_size/2), (m1_0[0], m1_0[1]+m_size/2),coordsA=ax.transData)
    k2 = patches.ConnectionPatch((m1_0[0]+m_size, m_size/2), (m2_0[0], m2_0[1]+m_size/2),coordsA=ax.transData)
    k3 = patches.ConnectionPatch((m2_0[0]+m_size, m_size/2), (m3_0[0], m3_0[1]),coordsA=ax.transData)
    k4 = patches.ConnectionPatch((m3_0[0], m3_0[1]+2*m_size), (m3_0[0]+spring_size, m3_0[1]+2*m_size),coordsA=ax.transData)
    k5 = patches.ConnectionPatch((m3_0[0], m3_0[1]+2*m_size), (m3_0[0], m3_0[1]+2*m_size+spring_size),coordsA=ax.transData)

    def init():
        ax.add_patch(m1)
        ax.add_patch(m2)
        ax.add_patch(m3)
        ax.add_patch(g1)
        ax.add_patch(g2)
        ax.add_patch(g3)
        ax.add_patch(k1)
        ax.add_patch(k2)
        ax.add_patch(k3)
        ax.add_patch(k4)
        ax.add_patch(k5)
        
        return m1,m2,m3,g1,g2,g3,k1,k2,k3,k4,k5,
    
    def animate(i):
        m1.set_xy([m1_0[0] + q[0][i], m1_0[1]])
        m2.set_xy([m2_0[0] + q[1][i], m2_0[1]])
        m3.xy1 = (m3_0[0] + q[2][i], m3_0[1] + q[3][i])
        m3.xy2 = (m3_0[0] + q[2][i] + 2*m_size*np.cos(q[4][i]-np.pi/2), 
                  m3_0[1]-2*m_size*np.sin(q[4][i]-np.pi/2))
        k1.xy2 = (m1_0[0] + q[0][i], m1_0[1]+m_size/2)
        k2.xy1 = (m1_0[0] + m_size + q[0][i], m_size/2)
        k2.xy2 = (m2_0[0] + q[1][i], m2_0[1]+m_size/2)
        k3.xy1 = (m2_0[0] + m_size + q[1][i], m_size/2)
        k3.xy2 = (m3_0[0] + q[2][i], m3_0[1] + q[3][i])
        k4.xy1 = m3.xy2
        k5.xy1 = m3.xy2

        g1.center = (m1_0[0]-spring_size + s*np.sin(t_num[i]*w), m1_0[0] + m_size/2)
        k1.xy1 = g1.center
        return m1,m2,m3,k1,k2,k3,k4,k5,g1
    
    anim = animation.FuncAnimation(fig, animate,
                                   init_func=init,
                                   frames=len(x),
                                   interval=500,
                                   blit=True)
    plt.close(fig)
    
    return HTML(anim.to_jshtml(fps=int(1/delta_t)))

In [ ]:
plot_gen_coords(q_num[0],t_num,w_value)

In [ ]:
animate_system(q_num[0],t_num,w_value,s_value)

### Damped

In [ ]:
# calculate first order damping coefficients beta_kk of the conservative system
beta_kk = np.array([(x.T).dot(C).dot(x) for x in eigmode_ud.T])
display(Math("""\mathbf{beta_{r}} = """ + vlatex(Matrix(beta_kk))))

# damping ratio  (4.54)
epsilon_r = (beta_kk/(2*eigfreq_ud*mu_r)).astype(float)
display(Math("""\mathbf{\epsilon_{r}} = """ + vlatex(Matrix(epsilon_r))))

# external forces from q matrix including damping
s,w = symbols('s w')
x0_def = s*sin(w*t)
ptD = qEq.xreplace({x0:x0_def}).doit()

# modal participation factor with damped p(t) (4.23)
phiD_r = []
for j in range(5):
    phiD = (eigmode_ud[:,j].T).dot(ptD) / mu_r[j]
    phiD_r.append(phiD[0])
display(Math("""\mathbf{\phi_{damped}}(t) = """ + vlatex(sp.Matrix(phiD_r))))


# inverse laplace transform hr from slides of lecture 14 from 2018
omega_dr = np.multiply(eigfreq_ud,np.sqrt(1-np.square(epsilon_r)))
hD_r = []
for j in range(5):
    hD = 1/omega_dr[j]*sp.exp(-epsilon_r[j]*eigfreq_ud[j]*t)*sp.sin(omega_dr[j]*t)
    hD_r.append(hD)

display(Math("""\mathbf{h_{damped}}(t) = """ + vlatex(sp.Matrix(hD_r))))


In [ ]:
w_value = 230
s_value = 0.0001
x0_values = {w:w_value,
              s:s_value}

t1 = 0
t2 = 0.5
delta_t = 0.001
t_num = np.linspace(t1,t2-delta_t,int((t2-t1)/delta_t))
phi_eval = Matrix(phiD_r).xreplace(values).xreplace(x0_values)
phi_num = np.array([phi_eval.xreplace({t:ti}) for ti in t_num]).T.astype(np.float64)[0]
h_num = np.array([Matrix(hD_r).xreplace({t:ti}) for ti in t_num]).T.astype(np.float64)[0]

# solution for eta_r with discrete convolution
etaD_r = []
etaDalt_r = []
for j in range(5):
    etaD = np.convolve(phi_num[j,:],h_num[j,:],mode='full')
    etaD_r.append(etaD)
etaD_r = np.array(etaD_r)[:len(t_num)]
display(Math("""\mathbf{\eta_{damped}}(t) = """ + vlatex(sp.Matrix(etaD_r))))


# system response q(t)
q_res_d = np.empty((5,))
for tj in range(len(t_num)):
    q_d = np.zeros(5,)
    for j in range(5):
        q_t = etaD_r[:,tj][j]*eigmode_ud[:,j]
        q_d = q_d + q_t
    q_res_d = np.column_stack((q_res_d, q_d))
q_res_d = np.asarray(q_res_d).astype(np.float64)[:,1:]
display(Math("""\mathbf{q_{damped}}(t) = """ + vlatex(sp.Matrix(q_res_d))))



In [ ]:
#q_res_d[:,:100].shape
plot_gen_coords(q_res_d,t_num,w_value)

In [ ]:
animate_system(q_res_d,t_num,w_value,s_value)